# Task 2: Handwritten Digits Recognition

Method 1: Feedforward Neural Network (30 points)

任务定义：利用前馈神经网络识别手写数字  
输入：手写数字图片  
输出：识别结果  
性能评价：准确率  
环境：python3.5  

In [6]:
# 导入所需要的包
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [13]:
# 定义三层全连接神经网络，每一层都是线性的
class simpleNet(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        #传入的参数分别为：输入的维度、第一层网络的神经元个数、第二层网络的神经元个数、第三层网络（输出层）的神经元个数
        super(simpleNet, self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
# 添加激活函数，改进网络的非线性
class Activation_Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Activation_Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
# 添加一个快速收敛的方法——批标准化
class Batch_Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Batch_Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim, n_hidden_1),nn.BatchNorm1d(n_hidden_1), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1, n_hidden_2),nn.BatchNorm1d(n_hidden_2), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [14]:
# 定义超参数
batch_size = 64
learning_rate = 1e-2
num_epoches = 1

In [17]:
if __name__ == '__main__':
    # 数据预处理
    data_tf = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
    # 下载训练集-MNIST手写数字训练集
    train_dataset = datasets.MNIST(root="./MNIST_data/", train=True, transform=data_tf, download=True)
    test_dataset = datasets.MNIST(root="./MNIST_data/", train=False, transform=data_tf)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # 选择并导入网络模型
    # (输入维度28x28，两个隐层分别是300和100，最后输出结果是10，因为一共有10个分类)
    model = simpleNet(28*28, 300, 100, 10)
    if torch.cuda.is_available():
        model = model.cuda()
    # 定义损失函数和优化函数
    criterion = nn.CrossEntropyLoss()#损失函数：损失函数交叉熵
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)# 优化函数1：随机梯度下降法
    optimizer_ = optim.Adam(model.parameters(), lr=learning_rate)# 优化函数2：Adam算法
    # 训练网络
    for epoch in range(50000):
        img, label = iter(train_loader).next()
        if torch.cuda.is_available():
            img = Variable(img.view(img.size(0), -1)).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img.view(img.size(0), -1))
            label = Variable(label)
        # 前向传播
        out = model(img)
        loss = criterion(out, label)
        # 反向传播
        optimizer.zero_grad()# 梯度归零
        loss.backward()
        optimizer.step()# 更新参数
        if(epoch + 1) % 1000 == 0:
            print('*'*10)
            print('epoch{}'.format(epoch+1))
            print('loss is {:.4f}'.format(loss.item()))
    # 测试网络
    model.eval()
    eval_loss = 0
    eval_acc = 0
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.item() * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.item()
    print('Test Loss:{:.6f}, Acc:{:.6f}'.format(eval_loss / (len(test_dataset)), eval_acc / (len(test_dataset))))

**********
epoch1000
loss is 0.4930
**********
epoch2000
loss is 0.1622
**********
epoch3000
loss is 0.2389
**********
epoch4000
loss is 0.2321
**********
epoch5000
loss is 0.1347
**********
epoch6000
loss is 0.3404
**********
epoch7000
loss is 0.4059
**********
epoch8000
loss is 0.1814
**********
epoch9000
loss is 0.2015
**********
epoch10000
loss is 0.3582
**********
epoch11000
loss is 0.1209
**********
epoch12000
loss is 0.3579
**********
epoch13000
loss is 0.3057
**********
epoch14000
loss is 0.2809
**********
epoch15000
loss is 0.3333
**********
epoch16000
loss is 0.2478
**********
epoch17000
loss is 0.1817
**********
epoch18000
loss is 0.2659
**********
epoch19000
loss is 0.2286
**********
epoch20000
loss is 0.2716
**********
epoch21000
loss is 0.1252
**********
epoch22000
loss is 0.2456
**********
epoch23000
loss is 0.2494
**********
epoch24000
loss is 0.3555
**********
epoch25000
loss is 0.3145
**********
epoch26000
loss is 0.0995
**********
epoch27000
loss is 0.2365
**********

Method 2: Convolutional Neural Network (30 points)

任务定义：利用卷积神经网络识别手写数字  
输入：手写数字图片  
输出：识别结果  
性能评价：准确率  
环境：python3.5  

In [20]:
# 导入所需要的包
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import time

In [26]:
#定义网络结构
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()

        # 由于MNIST为28x28， 而最初AlexNet的输入图片是227x227的。所以网络层数和参数需要调节
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) #AlexCONV1(3,96, k=11,s=4,p=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#AlexPool1(k=3, s=2)
        self.relu1 = nn.ReLU()

        # self.conv2 = nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)#AlexCONV2(96, 256,k=5,s=1,p=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)#AlexPool2(k=3,s=2)
        self.relu2 = nn.ReLU()


        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)#AlexCONV3(256,384,k=3,s=1,p=1)
        # self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)#AlexCONV4(384, 384, k=3,s=1,p=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)#AlexCONV5(384, 256, k=3, s=1,p=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#AlexPool3(k=3,s=2)
        self.relu3 = nn.ReLU()

        self.fc6 = nn.Linear(256*3*3, 1024)  #AlexFC6(256*6*6, 4096)
        self.fc7 = nn.Linear(1024, 512) #AlexFC6(4096,4096)
        self.fc8 = nn.Linear(512, 10)  #AlexFC6(4096,1000)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool3(x)
        x = self.relu3(x)
        x = x.view(-1, 256 * 3 * 3)#Alex: x = x.view(-1, 256*6*6)
        x = self.fc6(x)
        x = F.relu(x)
        x = self.fc7(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc8(x)
        x = self.dropout(x)
        return x

In [27]:
# 图片变换
transform = transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomGrayscale(),
                    transforms.ToTensor(),
])
transform1 = transforms.Compose([
                    transforms.ToTensor()
])

In [28]:
# 加载数据
trainset = torchvision.datasets.MNIST(root='./MNIST_data',train=True,download=True,transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,shuffle=True,num_workers=0)# windows下num_workers设置为0，不然有bug

testset = torchvision.datasets.MNIST(root='./MNIST_data',train=False,download=True,transform=transform1)
testloader = torch.utils.data.DataLoader(testset,batch_size=100,shuffle=False,num_workers=0)

In [29]:
# net
net = AlexNet()

# 损失函数:这里用交叉熵
criterion = nn.CrossEntropyLoss()

# 优化器 这里用SGD
optimizer = optim.SGD(net.parameters(),lr=1e-3, momentum=0.9)

# device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net.to(device)

AlexNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu2): ReLU()
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu3): ReLU()
  (fc6): Linear(in_features=2304, out_features=1024, bias=True)
  (fc7): Linear(in_features=1024, out_features=512, bias=True)
  (fc8): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.5)
)

In [31]:
# 训练
print("Start Training!")

#训练次数
num_epochs = 20 

for epoch in range(num_epochs):
    running_loss = 0
    batch_size = 100

    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('[%d, %5d] loss:%.4f'%(epoch+1, (i+1)*100, loss.item()))
print("Finished Traning")
#保存训练模型
torch.save(net, 'MNIST.pkl')
net = torch.load('MNIST.pkl')
#开始识别
with torch.no_grad():
    #在接下来的代码中，所有Tensor的requires_grad都会被设置为False
    correct = 0
    total = 0

    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        out = net(images)
        _, predicted = torch.max(out.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images:{}%'.format(100 * correct / total)) #输出识别准确率


Start Training!
[1, 60000] loss:2.2983
[2, 60000] loss:2.3027
[3, 60000] loss:2.2937
[4, 60000] loss:2.2547
[5, 60000] loss:0.6345
[6, 60000] loss:0.3230
[7, 60000] loss:0.3502
[8, 60000] loss:0.2666
[9, 60000] loss:0.1415
[10, 60000] loss:0.1994
[11, 60000] loss:0.2125
[12, 60000] loss:0.2834
[13, 60000] loss:0.1174
[14, 60000] loss:0.0457
[15, 60000] loss:0.0400
[16, 60000] loss:0.0501
[17, 60000] loss:0.0707
[18, 60000] loss:0.0499
[19, 60000] loss:0.0258
[20, 60000] loss:0.0714
Finished Traning
Accuracy of the network on the 10000 test images:97.03%



# Boston Housing Problem 

Method 2. Bayes classifier (20 points) 

任务定义：利用高斯贝叶斯模型来根据特征对波士顿房价分类  
输入：波士顿房子的特征    
输出：波士顿房子房价的类别    
性能评价：准确率  
环境：python3.5  

In [104]:
# 导入所需要的包
import sklearn
import pandas as pd
import warnings
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
warnings.simplefilter('ignore')

In [105]:
# 导入数据
data = pd.read_csv('./Boston.csv')
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATI,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [106]:
# 获取X和y
X = data.ix[:,:-1]
y = data['MEDV'] 
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATI,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48


In [107]:
y.describe()

count    506.000000
mean      22.532806
std        9.197104
min        5.000000
25%       17.025000
50%       21.200000
75%       25.000000
max       50.000000
Name: MEDV, dtype: float64

分为四个等级，断点为17、21和25

In [108]:
for i in range(len(y)):
    if y[i]<=17:
        y[i] = 0
    elif y[i]<=21:
        y[i] = 1
    elif y[i]<=25:
        y[i] = 2
    else:
        y[i] = 3
y

0      2.0
1      2.0
2      3.0
3      3.0
4      3.0
      ... 
501    2.0
502    1.0
503    2.0
504    2.0
505    0.0
Name: MEDV, Length: 506, dtype: float64

In [109]:
# 转换为numpy数组
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(506, 13)
(506,)


In [110]:
# 贝叶斯学习函数
def Bys_learn(X,y):
    model = GaussianNB()
    model.fit(X,y)
    return model

In [111]:
# demo
bys = Bys_learn(X,y)
bys

GaussianNB(priors=None, var_smoothing=1e-09)

In [112]:
# 贝叶斯预测函数
def Bys_predict(model,X):
    y = model.predict(X)
    return y

In [113]:
# 预测的结果
y = Bys_predict(bys,X)
y[:20]

array([3., 2., 3., 3., 3., 3., 2., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       1., 1., 1.])

In [114]:
# 划分数据集
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [115]:
# 训练
bys = Bys_learn(X_train,y_train)

In [116]:
# 测试
y_pred_test = Bys_predict(bys,X_test)
print(y_pred_test[:20])
print('Accuracy:',end = '')
print(accuracy_score(y_pred_test,y_test))

[2. 3. 0. 3. 1. 0. 2. 3. 0. 2. 0. 2. 1. 1. 0. 0. 0. 2. 2. 2.]
Accuracy:0.9411764705882353


Method 4: Neural Network for classification or regression (20 points) 

任务定义：利用神经网络来根据特征对波士顿房价回归  
输入：波士顿房子的特征    
输出：波士顿房子房价    
性能评价：均方差损失  
环境：python3.5  

In [117]:
# 导入所需要的包
from sklearn import preprocessing

In [129]:
# 载入数据，并预处理
X = np.array(data.ix[:,:-1])
y = np.array(data['MEDV'])
X = preprocessing.scale(X)
y = preprocessing.scale(y.reshape(-1,1))

In [135]:
# 定义超参数
data_size, D_input, D_output, D_hidden = X.shape[0], X.shape[1], 1, 50
lr = 1e-5
epoch = 20000

In [136]:
# sigmoid函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [137]:
# 权重w1和w2
w1 = np.random.randn(D_input, D_hidden)
w2 = np.random.randn(D_hidden, D_output)

In [138]:
# 训练
for i in range(epoch):
    # 前向传播
    h = np.dot(X, w1)
    h_ = sigmoid(h)
    y_ = np.dot(h, w2)
    # 打印误差
    mse = np.sum((y - y_) ** 2)/len(y_)
    if i % 1000 == 0:
        print('epoch: {} loss: {:.4f}'.format(i, mse))
    # 误差反向传播
    g_y_ = 2 * (y_ - y)
    g_w2 = np.dot(h_.T, g_y_)
    g_h_ = np.dot(g_y_, w2.T)
    g_h = g_h_ * sigmoid(h) * (1 - sigmoid(h))
    g_w1 = np.dot(X.T, g_h)
    # 参数更新
    w1 -= lr * g_w1
    w2 -= lr * g_w2

epoch: 0 loss: 484.0730
epoch: 1000 loss: 0.2754
epoch: 2000 loss: 0.2749
epoch: 3000 loss: 0.2747
epoch: 4000 loss: 0.2745
epoch: 5000 loss: 0.2743
epoch: 6000 loss: 0.2742
epoch: 7000 loss: 0.2742
epoch: 8000 loss: 0.2741
epoch: 9000 loss: 0.2742
epoch: 10000 loss: 0.2742
epoch: 11000 loss: 0.2742
epoch: 12000 loss: 0.2743
epoch: 13000 loss: 0.2744
epoch: 14000 loss: 0.2745
epoch: 15000 loss: 0.2746
epoch: 16000 loss: 0.2747
epoch: 17000 loss: 0.2748
epoch: 18000 loss: 0.2749
epoch: 19000 loss: 0.2750
